In [1]:
import spacy
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score, precision_score, recall_score, classification_report
import os
from tqdm import tqdm
import conllu
import tqdm
import torch
import json
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import functional
from functools import partial
from transformers import BertModel
from torch import nn
from transformers import BertTokenizer
import os
import pandas as pd




In [2]:
def transform_to_binary(set):
    binary_labels = []
    for label in set:
        if label == "O":
            binary_labels.append(0)
        else:
            binary_labels.append(1)
    return binary_labels

In [3]:
## Conllu stuff
def filter_tags(x):
    return x        

def convert_to_list_dict(path, file, lower=None):
    path = path.format(file)
    if lower == True:
        with open(path, encoding="UTF-8") as infile:
            lst = []
            tokens = list(conllu.parse_incr(infile))
            for sent in tokens:
                    dic = {
                    "idx": sent.metadata["sent_id"],
                    "text": sent.metadata["text"].lower(),
                    "tokens": [token["form"].lower() for token in sent],
                    "lemmas": [token["lemma"] for token in sent],
                    "pos_tags": [token["upos"] for token in sent],
                    "ner_tags": [filter_tags(token["misc"].get("name", "O")) for token in sent],
                }
                    lst.append(dic) 
            print("Converting {} to list of dictionaries\n     {} elements converted..".format(file, len(lst)))
    else:
        with open(path, encoding="UTF-8") as infile:
            lst = []
            tokens = list(conllu.parse_incr(infile))
            for sent in tokens:
                    dic = {
                    "idx": sent.metadata["sent_id"],
                    "text": sent.metadata["text"],
                    "tokens": [token["form"] for token in sent],
                    "lemmas": [token["lemma"] for token in sent],
                    "pos_tags": [token["upos"] for token in sent],
                    "ner_tags": [filter_tags(token["misc"].get("name", "O")) for token in sent],
                }
                    lst.append(dic) 
            print("Converting {} to list of dictionaries\n     {} elements converted..".format(file, len(lst)))
    return lst

In [4]:
path = r"C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\Results\NCRFpp\all_conllu/{0}.conllu"
file_list = ["no_bokmaal-ud-dev", "no_bokmaal-ud-test", "no_bokmaal-ud-train", "no_nynorsk-ud-dev", "no_nynorsk-ud-test", "no_nynorsk-ud-train"]
test_split_no = convert_to_list_dict(path, file_list[1], lower=True)
test_split_ny = convert_to_list_dict(path, file_list[4], lower=True)
TRUE_test_split_no = convert_to_list_dict(path, file_list[1])
TRUE_test_split_ny = convert_to_list_dict(path, file_list[4])



test_split = test_split_no + test_split_ny
gold_split = TRUE_test_split_no + TRUE_test_split_ny


Converting no_bokmaal-ud-test to list of dictionaries
     1939 elements converted..
Converting no_nynorsk-ud-test to list of dictionaries
     1511 elements converted..
Converting no_bokmaal-ud-test to list of dictionaries
     1939 elements converted..
Converting no_nynorsk-ud-test to list of dictionaries
     1511 elements converted..


In [5]:
nlp = spacy.load("nb_core_news_lg")



In [11]:
# implementing bindestrek fix

pred_sentences = []
for sen in test_split:
    doc = nlp(" ".join(sen["tokens"]))
    pred_sent = []
    for token in doc:
        ## Checking if "-" (bindestrek) in token. Such as in "Sør-Trøndelag"
        if token.ent_type_ != "":
            print(doc)
            print(token.ent_type_) 
            if "-" in token.text:
                s = []
                for i, c in enumerate(token.text):
                    if i == 0:
                        s.append(c.capitalize())
                    elif c == "-":
                        s.append("-")
                    elif token.text[i-1] == "-":
                        s.append(c.capitalize())
                    else:
                        s.append(c)
                pred_sent.append("".join(s))
                continue
                
            pred_sent.append(token.text.capitalize())
        else:
            pred_sent.append(token.text)
    pred_sentences.append(" ".join(pred_sent))

dette er som george orwells nytale , av samme logiske gehalt som " fred er krig " , " stillhet er larm " , " lys er mørke " .
PER
dette er som george orwells nytale , av samme logiske gehalt som " fred er krig " , " stillhet er larm " , " lys er mørke " .
PER
i diktet " om å vokse nedover " skriver rolf jacobsen :
PER
i diktet " om å vokse nedover " skriver rolf jacobsen :
PER
i new york er du bare 10 cm .
GPE_LOC
i new york er du bare 10 cm .
GPE_LOC
i london og singapore kanskje en engelsk fot .
GPE_LOC
i london og singapore kanskje en engelsk fot .
GPE_LOC
hvordan skal det gå med eu ?
GPE_ORG
tyrkia .
GPE_LOC
det skrives mye om hvordan øst-asia stiger opp og usa faller ned .
GPE_LOC
men i den nye internasjonale orden er det europa og eu som har de største problemene .
GPE_ORG
men i den nye internasjonale orden er det europa og eu som har de største problemene .
GPE_ORG
i 1945 var storbritannia og kanskje også frankrike stormakter .
GPE_ORG
i 1945 var storbritannia og kanskje også fr

KeyboardInterrupt: 

In [49]:
pred_sentences_list = []
for sentence in pred_sentences:
    pred_sentences_list.append(sentence.split())

In [50]:
pred_annotations = []
for sentence in pred_sentences:
    annoation = []
    for token in sentence.split():
        if token[0].isupper(): # Checks if token has captial letter
            annoation.append(1)
        else:
            annoation.append(0) # Adds 0 if token has lower 
    pred_annotations.append((annoation))
            

In [51]:
# implementing bindestrek fix

gold_label_sentences = []
for sen in gold_split:
    doc = nlp(" ".join(sen["tokens"]))
    pred_sent = []
    for token in doc:
        ## Checking if "-" (bindestrek) in token. Such as in "Sør-Trøndelag"
        if token.ent_type_ != "": 
            if "-" in token.text:
                s = []
                for i, c in enumerate(token.text):
                    if i == 0:
                        s.append(c.capitalize())
                    elif c == "-":
                        s.append("-")
                    elif token.text[i-1] == "-":
                        s.append(c.capitalize())
                    else:
                        s.append(c)
                pred_sent.append("".join(s))
                continue
                
            pred_sent.append(token.text.capitalize())
        else:
            pred_sent.append(token.text)
    gold_label_sentences.append((pred_sent))

In [52]:
gold_labels = []
for sentence in gold_label_sentences:
    label = []
    for token in sentence:
        if token[0].isupper(): # Checks if token has captial letter
            label.append(1)
        else:
            label.append(0) # Adds 0 if token has lower 
    gold_labels.append((label))
            

In [53]:
# gold_labels = []
# for tags in test_split:
#     gold_labels.append(tags["ner_tags"])

In [54]:
# y_true = [item for sublist  in gold_labels for item in sublist]
# y_true = transform_to_binary(y_true)

In [55]:
y_pred = [item for sublist  in pred_annotations for item in sublist]
y_true = [item for sublist  in gold_labels for item in sublist]

In [56]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [54746, 54749]

In [57]:
"""Making list of lists preds, same len as NorNE gold labels"""
""" test_dataset_ner_labels should be test_dataset.ner_labels """

def split_list_preds(preds, test_dataset_ner_labels):
    split_list_preds = []
    start = 0

    for sublist in test_dataset_ner_labels:
        end = start + len(sublist)
        split_list_preds.append(preds[start:end])
        start = end
    return split_list_preds

In [ ]:
""" 
Because of how spacy tokenizes words, there is a slight discrepancy in the token count between gold and pred sets.
To counteract this, the 3 given sentences are dropped from both sets.
"""




for i, (t, p) in enumerate(zip(gold_labels, pred_annotations)):
    if len(t)!=len(p):
        print(f"{i}: {p} pred_sentences_list: {pred_sentences_list[i]}")
        print(f"true: {t} gold_label_sentences: {gold_label_sentences[i]}")
        print()


preds_copy = pred_annotations
gold_copy = gold_labels

index_remove_numbers = [1063, 1270, 1813]
for i, n in enumerate(index_remove_numbers):
    preds_copy.pop(n-i)
    gold_copy.pop(n-i)

In [64]:
for i, (t, p) in enumerate(zip(gold_copy, preds_copy)):
    if len(t)!=len(p):
        print(f"{i}: {p} pred_sentences_list: {pred_sentences_list[i]}")
        print(f"true: {t} gold_label_sentences: {gold_label_sentences[i]}")
        print()

In [66]:
y_pred = [item for sublist  in gold_copy for item in sublist]
y_true = [item for sublist  in preds_copy for item in sublist]

In [67]:
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

[[48032  4776]
 [   35  1876]]
              precision    recall  f1-score   support

           0       1.00      0.91      0.95     52808
           1       0.28      0.98      0.44      1911

    accuracy                           0.91     54719
   macro avg       0.64      0.95      0.70     54719
weighted avg       0.97      0.91      0.93     54719

